<strong>PyChemkin</strong>'s <i><u>detonation</u></i> <em>Mixture</em> method is a conveneint tool to determine the Chapman-Jouguet state and the detonation wave speed of a combustible mixture. This tutorial utilizes the <i><u>detonation</u></i> method to predict the detonation wave speeds of a natural gas-air mixture at various <u>initial pressures</u> and compare the <u>ideal-gas</u> and the <u>real-gas</u> results against the measurements. 

To start a <strong>PyChemkin</strong> project, you always have to import these Python packages: <code>os</code>, <code>chemkin</code>, and <code>numppy</code>. The <code>matplotlib</code> package is also imported for plotting purposes.

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
%matplotlib inline
import matplotlib.pyplot as plt           # plotting

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<h5>Set up the <em>Chemistry Set</em></h5>Create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the C2_NOx_SRK mechanism for C2 hydrocarbons combustion is used to create the <code>MyGasMech</code> <em>Chemistry</em> object.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# create a chemistry set based on C2_NOx using an alternative method
MyMech = ck.Chemistry(label='C2 NOx')
# set mechanism input files individually
# this mechanism file contains all the necessary thermodynamic and transport data
# therefore no need to specify the therm and the tran data files
MyMech.chemfile = mechanism_dir+'\\C2_NOx_SRK.inp'
# preprocess the 2nd mechanism files
iError = MyMech.preprocess()

<h5>Define the fuel-air mixture</h5>Use the <i><u>equivalence ratio</u></i> method to create a stoiciometric fuel-air mixture <code>premixed</code>.

In [ ]:
# create the fuel mixture
fuel = ck.Mixture(MyMech)
# set mole fraction
fuel.X = [('CH4', 0.8), ('C2H6', 0.2)]
fuel.temperature = 290.0
fuel.pressure = 40.0 * ck.Patm
# create the air mixture
air = ck.Mixture(MyMech)
# set mass fraction
air.X = [('O2', 0.21), ('N2', 0.79)]
air.temperature = fuel.temperature
air.pressure = fuel.pressure
# create the initial mixture
# create the premixed mixture to be defined by equivalence ratio
premixed = ck.Mixture(MyMech)
# products from the complete combustion of the fuel mixture and air
products = ['CO2', 'H2O', 'N2']
# species mole fractions of added/inert mixture. can also create an additives mixture here
add_frac = np.zeros(MyMech.KK, dtype=np.double)   # no additives: all zeros
iError = premixed.XbyEquivalenceRatio(MyMech, fuel.X, air.X, add_frac, products, equivalenceratio=1.0)
if iError > 0:
        raise RuntimeError

Verify the fuel-air mixture composition.

In [ ]:
# list the composition of the premixed mixture
premixed.listcomposition(mode='mole')

<h5>Find the Chapman-Jouguet state and the detonation wave speed of the fuel-air mixture</h5>Use the <em>Mixture</em> <code>detonation</code> method to find the C-J state and the detonation wave speed of the fuel-air mixture with the initial mixture pressure inceasing from 40 to 80 atm. The initial mixture temperature is kept at 290K. The <code>detonation</code> method will return two objects: the <code>speed</code> <i>list</i> containing the <u>speed of sound</u> and the <u>detonation wave speed</u> at the C-J state; the <code>CJState</code> <em>Mixture</em> object containing the mixture properties. For instance, you can get the mixture pressure at the C-J state using the method <code>CJState.pressure</code>. Note that by default <em>Chemkin</em> variables are in the <u>cgs</u> units.<br><hr>You can check out the input and the output parameters of the <code>detonation</code> method by issuing command <code>ck.Mixture.detonation.__doc__</code> or <code>ck.help('equilibrium')</code> at the Python prompt. 

In [ ]:
# set up parameter study of detonation wave speed with respect to pressure
points = 5
dpres = 10.0 * ck.Patm
pres = fuel.pressure
P = np.zeros(points, dtype=np.double)
Det_ig = np.zeros_like(P, dtype=np.double)
premixed.pressure = pres
premixed.temperature = fuel.temperature
# start of pressure loop
for i in range(points):
    # compute the C-J state corresponding to the initial mixture
    speed, CJstate = ck.detonation(premixed)
    # update plot data
    # convert pressure to atm
    P[i] = pres / ck.Patm
    # convert speed to m/sec
    Det_ig[i] = speed[1] / 1.0e2
    # update pressure value
    pres += dpres
    premixed.pressure = pres

<h5>Switch to the real gas EOS model</h5>Use the <code>userealgascubicEOS</code> method to turn ON the <em>Chemkin</em> real-gas EOS model. For more information either type <code>help('real gas')</code> for the real-gas model usage in <strong>PyChemkin</strong> or type <code>help('manuals')</code> to access the on-line <em>Chemkin</em> <u>Theory</u> manual for descriptions of the real-gas EOS models. 

In [ ]:
# turn on real-gas cubic equation of state
premixed.userealgascubicEOS()
# set mixture mixing rule to Van der Waals (default)
# premixed.setrealgasmixingrule(rule=0)

<h5>Find the Chapman-Jouguet state and the detonation wave speed of the <u>real gas</u> fuel-air mixture</h5>The steps are exactly the same as the ones for the ideal gas mixture. The speed values from <code>detonation</code> method are suppressed by setting the <code>ck.verbose</code> value to <i>False</i>.  

In [ ]:
# restart the calculation with real-gas EOS
premixed.pressure = fuel.pressure
pres = fuel.pressure
P[:] = 0.0e0
Det_rg = np.zeros_like(P, dtype=np.double)
# set verbose mode to false to turn OFF extra printouts
ck.verbose = False
# start of pressure loop
for i in range(points):
    # compute the C-J state corresponding to the initial mixture
    speed, CJstate = ck.detonation(premixed)
    # update plot data
    P[i] = pres / ck.Patm
    Det_rg[i] = speed[1] / 1.0e2
    # update pressure value
    pres += dpres
    premixed.pressure = pres

<h5>Set up the experimental data values</h5>Just enter data to the lists.

In [ ]:
# experimental data
P_data = [44.1, 50.6, 67.2, 80.8]
Det_data = [1950.0, 1970.0, 2000.0, 2020.0]

<h5>Compare the detonation wave speeds</h5>Because of the relatively high pressures, the differences in the predicted detonation wave speeds between the ideal gas and the real-gas models can be significant.

In [ ]:
# create plot for ideal gas results
plt.plot(P, Det_ig, 'bo--', label='ideal gas', markersize=5, fillstyle='none')
# create plot for real gas results
plt.plot(P, Det_rg, 'r^-', label='real-gas', markersize=5, fillstyle='none')
# plot data
plt.plot(P_data, Det_data, 'gD:', label='data', markersize=4)
#
plt.xlabel('Pressure [atm]')
plt.ylabel('Detonation wave speed [m/sec]')
plt.legend(loc='upper left')
plt.show()